## **LAB 02**

Search in AI- using A* algorithm

I have implemented a simple **Pac-Man Game** with A* Algorithm.

The game is played on a grid with obstacles, and the game will end when:

*   Pac-Man collects all pellets (win condition).
*   A ghost catches Pac-Man (lose condition).

**How the Game works:**

  *   Pac-Man starts at the top-left corner of the grid (0,0).
  * Pellets (O) are randomly placed on the grid, and Pac-Man uses A* to find the nearest pellet.
  * Ghosts (G) are also randomly placed on the grid, and they chase Pac-Man using the A* algorithm.






In [ ]:
import heapq
import random
import math

# Create the grid for the Pac-Man game
def create_pacman_grid(size, num_pellets):
    grid = [[' ' for _ in range(size)] for _ in range(size)]
    grid[0][0] = 'P'  # Pac-Man starts at the top-left corner (0, 0)

    pellets = []
    while len(pellets) < num_pellets:
        pellet_x, pellet_y = random.randint(0, size - 1), random.randint(0, size - 1)
        if grid[pellet_x][pellet_y] == ' ' and (pellet_x, pellet_y) != (0, 0):
            grid[pellet_x][pellet_y] = 'O'  # 'O' represents pellets
            pellets.append((pellet_x, pellet_y))

    return grid, pellets  # Return the grid and list of pellet positions

# Add ghosts to the grid
def add_ghosts(grid, num_ghosts):
    size = len(grid)
    ghosts = []
    while len(ghosts) < num_ghosts:
        ghost_x, ghost_y = random.randint(0, size - 1), random.randint(0, size - 1)
        if grid[ghost_x][ghost_y] == ' ':
            grid[ghost_x][ghost_y] = 'G'  # 'G' represents a ghost
            ghosts.append((ghost_x, ghost_y))

    return ghosts

# Heuristic function for A*
def heuristic(a, b):
    return math.sqrt((a[0] - b[0])**2 + (a[1] - b[1])**2)

# A* algorithm function to find the shortest path
def a_star(grid, start, goal):
    open_list = []
    heapq.heappush(open_list, (0, start))

    g_score = {start: 0}
    parent = {start: None}

    directions = [(-1, 0), (1, 0), (0, -1), (0, 1)]  # Up, down, left, right

    while open_list:
        _, current = heapq.heappop(open_list)

        if current == goal:
            break

        for direction in directions:
            next_x = current[0] + direction[0]
            next_y = current[1] + direction[1]
            next_state = (next_x, next_y)

            if 0 <= next_x < len(grid) and 0 <= next_y < len(grid[0]) and grid[next_x][next_y] != 'X':
                tentative_g_score = g_score[current] + 1

                if next_state not in g_score or tentative_g_score < g_score[next_state]:
                    g_score[next_state] = tentative_g_score
                    f_score = tentative_g_score + heuristic(next_state, goal)
                    heapq.heappush(open_list, (f_score, next_state))
                    parent[next_state] = current

    path = []
    current = goal
    while current is not None:
        path.append(current)
        current = parent[current]
    path.reverse()

    return path

# Function to print the grid with Pac-Man, pellets, ghosts, and path marked
def print_grid(grid):
    print("\nGame Grid:")
    print('-' * (len(grid) * 4 + 1))
    for row in grid:
        print('| ' + ' | '.join(row) + ' |')
        print('-' * (len(grid) * 4 + 1))

# Function to check if Pac-Man has won by collecting all pellets
def check_win(pellets):
    return len(pellets) == 0

# Main game function
def run_pacman_game():
    size = int(input("Enter the grid size (e.g., 6 for a 6x6 grid): "))
    num_pellets = int(input(f"Enter the number of pellets (less than {size * size - 2}): "))
    num_ghosts = int(input(f"Enter the number of ghosts (less than {size * size - 2}): "))

    # Create grid, place pellets, and add ghosts
    grid, pellets = create_pacman_grid(size, num_pellets)
    ghosts = add_ghosts(grid, num_ghosts)

    pacman_pos = (0, 0)  # Pac-Man starts at the top-left corner

    while True:
        print_grid(grid)

        # Pac-Man's move: Find the nearest pellet
        nearest_pellet = min(pellets, key=lambda pellet: heuristic(pacman_pos, pellet))
        pacman_path = a_star(grid, pacman_pos, nearest_pellet)

        if pacman_path:
            pacman_pos = pacman_path[1]  # Move Pac-Man to the next step
            grid[pacman_pos[0]][pacman_pos[1]] = 'P'  # Update Pac-Man's position on the grid
            grid[0][0] = ' '  # Clear previous Pac-Man position

            # If Pac-Man reaches a pellet
            if pacman_pos == nearest_pellet:
                pellets.remove(nearest_pellet)
                grid[pacman_pos[0]][pacman_pos[1]] = 'P'  # Mark Pac-Man's position

        # Ghosts' move: Each ghost moves toward Pac-Man
        for i, ghost_pos in enumerate(ghosts):
            ghost_path = a_star(grid, ghost_pos, pacman_pos)
            if ghost_path:
                next_ghost_pos = ghost_path[1]
                grid[ghost_pos[0]][ghost_pos[1]] = ' '  # Clear old ghost position
                ghosts[i] = next_ghost_pos
                grid[next_ghost_pos[0]][next_ghost_pos[1]] = 'G'  # Update ghost's new position

            # Check if a ghost catches Pac-Man
            if ghosts[i] == pacman_pos:
                print("Pac-Man was caught by a ghost! Game over!")
                return

        # Check if Pac-Man wins by collecting all pellets
        if check_win(pellets):
            print("Pac-Man collected all the pellets! You win!")
            break

if __name__ == "__main__":
    run_pacman_game()


Enter the grid size (e.g., 6 for a 6x6 grid): 4
Enter the number of pellets (less than 14): 7
Enter the number of ghosts (less than 14): 8

Game Grid:
-----------------
| P | G | O | G |
-----------------
| O | O | G | G |
-----------------
| O | G | O | G |
-----------------
| G | G | O | O |
-----------------

Game Grid:
-----------------
| G |   | G |   |
-----------------
| P | G |   |   |
-----------------
| G |   | G |   |
-----------------
|   |   | O | O |
-----------------
Pac-Man was caught by a ghost! Game over!
